In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import statistics
import os
import re
###############################################
from peakutils import indexes
from peakutils import baseline
from scipy.signal import find_peaks as fp
from scipy.signal import savgol_filter 
###############################################
from bokeh.plotting import figure , show
from bokeh.models import Range1d

from pybaselines import whittaker as pl

from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [3]:
def load_data(folder_path):
    # List to store DataFrames for intensity columns
    intensity_dfs = []

    # List to store file names
    file_names = []

    # Get a list of .txt files in the folder
    txt_files = [file_name for file_name in os.listdir(folder_path) if file_name.endswith('.txt')]

    # Sort the .txt files based on their numerical order
    txt_files.sort(key=lambda x: int(re.search(r'_(\d+)\.txt', x).group(1)))

    # Read the wavelength values from the first file
    first_file_path = os.path.join(folder_path, txt_files[0])
    wavelength_df = pd.read_csv(first_file_path, header=None, delimiter=';', usecols=[0], names=['wavelength'])

    # Loop through each file in ascending order
    for file_name in txt_files:
        file_path = os.path.join(folder_path, file_name)
        
        # Read intensity values from each file into a DataFrame
        intensity_df = pd.read_csv(file_path, header=None, delimiter=';', usecols=[1], names=['intensity'])
        
        # Store intensity DataFrame
        intensity_dfs.append(intensity_df)
        
        # Store file name
        file_names.append(os.path.splitext(file_name)[0])

    # Concatenate intensity DataFrames
    result_df = pd.concat(intensity_dfs, axis=1)

    # Add the wavelength column to the result DataFrame
    result_df = pd.concat([wavelength_df, result_df], axis=1)

    # Rename the columns with file names
    result_df.columns = ['wavelength'] + file_names

    return result_df


# LOADING THE DATAFRAME

Please note that the '-x' behind the the name of the sample is cleanin shots , where -1 -> correspond to 0 cleaning shots , -2 -> correspond to 5 cleaning shots , -3 -> 10 cleaning shots , ------ , -12 -> 55 cleaning shots

In [4]:
df_raw = load_data('rLIBS_Spectra/Depth_Profiling/Polish/CR300LA')
df_raw

,wavelength,CR300LA_1,CR300LA_2,CR300LA_3,CR300LA_4,CR300LA_5,CR300LA_6,CR300LA_7,CR300LA_8,CR300LA_9,...,CR300LA_11,CR300LA_12,CR300LA_13,CR300LA_14,CR300LA_15,CR300LA_16,CR300LA_17,CR300LA_18,CR300LA_19,CR300LA_20
0,200.052741,3.000000,-3.000000,-6.000000,6.000000,8.000000,-13.000000,4.000000,15.000000,-4.000000,...,-21.000000,14.000000,-12.000000,-14.000000,-22.000000,-4.000000,7.000000,-1.000000,2.000000,7.000000
1,200.056350,0.000000,7.000000,1.000000,-2.000000,6.000000,-1.000000,17.000000,11.000000,10.000000,...,11.000000,3.000000,-17.000000,-5.000000,2.000000,12.000000,-8.000000,12.000000,-11.000000,-7.000000
2,200.059960,-8.000000,0.000000,-4.000000,14.000000,4.000000,10.000000,15.000000,-9.000000,-5.000000,...,21.000000,-6.000000,-3.000000,3.000000,2.000000,3.000000,-10.000000,3.000000,-2.000000,-5.000000
3,200.063569,8.000000,-8.000000,24.000000,1.000000,1.000000,-13.000000,-14.000000,-5.000000,12.000000,...,1.000000,15.000000,-2.000000,-2.000000,10.000000,8.000000,-5.000000,19.000000,6.000000,1.000000
4,200.067178,-27.000000,7.000000,10.000000,-10.000000,-17.000000,29.000000,9.000000,-5.000000,-9.000000,...,8.000000,-10.000000,1.000000,-6.000000,-8.000000,-21.000000,2.000000,10.000000,9.000000,-12.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92088,963.320605,0.319491,2.836877,3.987383,1.442479,8.935525,7.555236,-4.102325,-12.851778,12.853964,...,7.542619,5.851679,20.807967,21.795351,8.217166,0.488476,10.910292,1.442579,6.112657,-11.680509
92089,963.336411,5.220267,2.834986,19.834986,-5.438693,7.555430,12.267801,-3.561307,-1.553234,0.049375,...,-1.888397,11.881422,4.109762,11.439791,9.895372,17.836084,11.783770,-14.592696,17.940711,-5.662996
92090,963.352216,3.959889,16.204770,-2.715007,1.425801,5.072634,18.127676,12.093356,13.667554,-12.040111,...,-4.112746,15.517826,-2.602262,-7.618770,2.553477,-3.875453,14.261143,1.703206,15.606722,-18.814620
92091,963.368021,-5.682138,-31.125506,-3.604756,6.206627,22.501320,8.701949,6.209985,-1.586273,-17.194386,...,-29.031197,-0.169187,12.258581,1.827455,-23.857568,17.090951,-28.421526,-0.876051,-23.558190,-4.527846


In [5]:
def baseline_correction(df):
    """
    Perform baseline correction on the intensity columns of the input DataFrame and create a new DataFrame with corrected values.
    
    Parameters:
        df (DataFrame): Input DataFrame containing the wavelength and intensity columns.
        
    Returns:
        DataFrame: New DataFrame with baseline-corrected intensity columns and the same wavelength column as the input DataFrame.
    """
    # Copy the 'wavelength' column from the input DataFrame
    new_df = pd.DataFrame({'wavelength': df['wavelength']})
    
    # Perform baseline correction for each intensity column and add them to the new DataFrame
    for col in df.columns[1:]:  # Exclude the 'wavelength' column
        baseline, _ = pl.airpls(df[col],lam=0.1)
        corrected_values = df[col] - baseline
        new_df[col] = corrected_values
    
    return new_df

In [6]:
df_baselinecorrected =  df_raw
df_baselinecorrected

,wavelength,CR300LA_1,CR300LA_2,CR300LA_3,CR300LA_4,CR300LA_5,CR300LA_6,CR300LA_7,CR300LA_8,CR300LA_9,...,CR300LA_11,CR300LA_12,CR300LA_13,CR300LA_14,CR300LA_15,CR300LA_16,CR300LA_17,CR300LA_18,CR300LA_19,CR300LA_20
0,200.052741,3.000000,-3.000000,-6.000000,6.000000,8.000000,-13.000000,4.000000,15.000000,-4.000000,...,-21.000000,14.000000,-12.000000,-14.000000,-22.000000,-4.000000,7.000000,-1.000000,2.000000,7.000000
1,200.056350,0.000000,7.000000,1.000000,-2.000000,6.000000,-1.000000,17.000000,11.000000,10.000000,...,11.000000,3.000000,-17.000000,-5.000000,2.000000,12.000000,-8.000000,12.000000,-11.000000,-7.000000
2,200.059960,-8.000000,0.000000,-4.000000,14.000000,4.000000,10.000000,15.000000,-9.000000,-5.000000,...,21.000000,-6.000000,-3.000000,3.000000,2.000000,3.000000,-10.000000,3.000000,-2.000000,-5.000000
3,200.063569,8.000000,-8.000000,24.000000,1.000000,1.000000,-13.000000,-14.000000,-5.000000,12.000000,...,1.000000,15.000000,-2.000000,-2.000000,10.000000,8.000000,-5.000000,19.000000,6.000000,1.000000
4,200.067178,-27.000000,7.000000,10.000000,-10.000000,-17.000000,29.000000,9.000000,-5.000000,-9.000000,...,8.000000,-10.000000,1.000000,-6.000000,-8.000000,-21.000000,2.000000,10.000000,9.000000,-12.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92088,963.320605,0.319491,2.836877,3.987383,1.442479,8.935525,7.555236,-4.102325,-12.851778,12.853964,...,7.542619,5.851679,20.807967,21.795351,8.217166,0.488476,10.910292,1.442579,6.112657,-11.680509
92089,963.336411,5.220267,2.834986,19.834986,-5.438693,7.555430,12.267801,-3.561307,-1.553234,0.049375,...,-1.888397,11.881422,4.109762,11.439791,9.895372,17.836084,11.783770,-14.592696,17.940711,-5.662996
92090,963.352216,3.959889,16.204770,-2.715007,1.425801,5.072634,18.127676,12.093356,13.667554,-12.040111,...,-4.112746,15.517826,-2.602262,-7.618770,2.553477,-3.875453,14.261143,1.703206,15.606722,-18.814620
92091,963.368021,-5.682138,-31.125506,-3.604756,6.206627,22.501320,8.701949,6.209985,-1.586273,-17.194386,...,-29.031197,-0.169187,12.258581,1.827455,-23.857568,17.090951,-28.421526,-0.876051,-23.558190,-4.527846


# Normalization of the Spectra

Focusing on two main Normalization :- 

1) Normalization by Max Intensity of the Spectra
2) Normalization by the Matrix Peak i.e Fe Peak 259.90 - 259.8999999999988 , 238.16666666666336 , 263.10000000000133

In [7]:
# Normalization by the Maximum Intensity

def normalize_intensities(df):
    # Get the maximum intensity for each column starting from column 2
    max_intensities = df.iloc[:, 1:].max()
    max_wavelengths = df.iloc[:, 0][df.iloc[:, 1:].idxmax()]
    # Normalize intensities for each column
    for col_num in range(1, len(df.columns)):
        df.iloc[:, col_num] /= max_intensities[col_num - 1]
    
    return df


In [8]:
# Normalization bby SNV

def standard_normal_variate_normalization(df):
    # Selecting only the intensity columns
    intensities = df.iloc[:, 1:]

    # Applying standard normal variate normalization
    normalized_intensities = (intensities - intensities.mean()) / intensities.std()

    # Combining wavelength column with normalized intensities
    normalized_df = pd.concat([df.iloc[:, 0], normalized_intensities], axis=1)

    return normalized_df

In [9]:
# Normalization by the Matrix Fe Peak i.e Fe Peak 438.35

# Lets try to locate the Matrix Peak first

Matrix_Wavelength_Min = 438.3
Matrix_Wavelength_Max = 438.4

Matrix_df = df_baselinecorrected[(df_baselinecorrected['wavelength'] >= Matrix_Wavelength_Min) & (df_baselinecorrected['wavelength'] <= Matrix_Wavelength_Max)]


Selected_Matrix_Plot = figure(title='Selected Matrix Peak', x_axis_label='Wavelength', y_axis_label='Intensity', width=600, height=500)

# Loop through each column (starting from the second one)
for col_num, col_name in enumerate(Matrix_df.columns[1:], start=1):
    intensity_values = Matrix_df[col_name]
    Selected_Matrix_Plot.line(Matrix_df['wavelength'], intensity_values, line_width=2, color="red")

show(Selected_Matrix_Plot)
# Matrix_df

In [10]:
# def normalize_data(matrix_df, baseline_df):
#     normalized_df = pd.DataFrame() #crete a df
#     normalized_df['wavelength'] = baseline_df['wavelength'] #add first column as wavelength

#     for sample_col in matrix_df.columns[1:]:
#         peaks, _ = fp(matrix_df[sample_col], prominence=100) #finds peak from matrix_df
#         return_intensities = matrix_df[sample_col].iloc[peaks] #gets the intensity for that matrix peak
#         corresponding_col = sample_col
#         normalized_df[sample_col] = baseline_df[corresponding_col] / return_intensities.values[0] #create a columns in normalized_df , by dividing the baseline_df by the intensities of the matrix peak 

#     return normalized_df

In [11]:
def normalize_data(matrix_df, baseline_df):
    normalized_df = pd.DataFrame()  # create a df
    normalized_df['wavelength'] = baseline_df['wavelength']  # add first column as wavelength

    for sample_col in matrix_df.columns[1:]:
        max_intensity = matrix_df[sample_col].max()  # find the maximum intensity in the column
        corresponding_col = sample_col
        normalized_df[sample_col] = baseline_df[corresponding_col] / max_intensity  # create a column in normalized_df by dividing the baseline_df by the maximum intensity

    return normalized_df


In [12]:
# #NORMALIZATION BY MATRIX PEAK
df_normalized = normalize_data(matrix_df=Matrix_df , baseline_df=df_baselinecorrected)
df_normalized


,wavelength,CR300LA_1,CR300LA_2,CR300LA_3,CR300LA_4,CR300LA_5,CR300LA_6,CR300LA_7,CR300LA_8,CR300LA_9,...,CR300LA_11,CR300LA_12,CR300LA_13,CR300LA_14,CR300LA_15,CR300LA_16,CR300LA_17,CR300LA_18,CR300LA_19,CR300LA_20
0,200.052741,0.016909,-0.001883,-0.004797,0.006994,0.010249,-0.020124,0.007663,0.024884,-0.007120,...,-0.014487,0.013094,-0.008906,-0.010648,-0.018848,-0.003384,0.006929,-0.001162,0.003121,0.002940
1,200.056350,0.000000,0.004394,0.000799,-0.002331,0.007687,-0.001548,0.032566,0.018248,0.017800,...,0.007588,0.002806,-0.012617,-0.003803,0.001713,0.010151,-0.007919,0.013948,-0.017168,-0.002940
2,200.059960,-0.045092,0.000000,-0.003198,0.016318,0.005125,0.015480,0.028735,-0.014931,-0.008900,...,0.014487,-0.005612,-0.002227,0.002282,0.001713,0.002538,-0.009898,0.003487,-0.003121,-0.002100
3,200.063569,0.045092,-0.005021,0.019187,0.001166,0.001281,-0.020124,-0.026819,-0.008295,0.021361,...,0.000690,0.014029,-0.001484,-0.001521,0.008567,0.006767,-0.004949,0.022084,0.009364,0.000420
4,200.067178,-0.152185,0.004394,0.007994,-0.011656,-0.021779,0.044892,0.017241,-0.008295,-0.016020,...,0.005519,-0.009353,0.000742,-0.004563,-0.006854,-0.017765,0.001980,0.011623,0.014047,-0.005040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92088,963.320605,0.001801,0.001781,0.003188,0.001681,0.011448,0.011696,-0.007859,-0.021320,0.022881,...,0.005203,0.005473,0.015443,0.016577,0.007040,0.000413,0.010799,0.001677,0.009540,-0.004905
92089,963.336411,0.029424,0.001779,0.015857,-0.006339,0.009680,0.018991,-0.006822,-0.002577,0.000088,...,-0.001303,0.011112,0.003050,0.008701,0.008478,0.015088,0.011664,-0.016961,0.028001,-0.002378
92090,963.352216,0.022320,0.010172,-0.002170,0.001662,0.006499,0.028062,0.023167,0.022674,-0.021432,...,-0.002837,0.014513,-0.001931,-0.005795,0.002188,-0.003278,0.014116,0.001980,0.024358,-0.007902
92091,963.368021,-0.032027,-0.019537,-0.002882,0.007234,0.028827,0.013471,0.011896,-0.002632,-0.030607,...,-0.020027,-0.000158,0.009098,0.001390,-0.020440,0.014458,-0.028132,-0.001018,-0.036768,-0.001902


In [13]:
# # Normalization by SNV

# df_normalized = standard_normal_variate_normalization(df=df_baselinecorrected)
# df_normalized


In [14]:
# # NORMALIZATION BY MAXIMUM INTENSITYT

# df_normalized = normalize_intensities(df_baselinecorrected)
# df_normalized

# Plotting the Zinc Peak
The Zn peaks for LIBS are located at Zn 328.23 , Zn 468.02

In [18]:
Zn1_Peak_Min = 328.0,  #For CR300LA & CR240LA - In complience with NIST Database
Zn1_Peak_Max = 328.5, #For CR300LA & CR240LA - In complience with NIST Database

Zn2_Peak_Min = 467.8 , #For CR300LA & CR240LA - In complience with NIST Database
Zn2_Peak_Max = 468.2 , #For CR300LA & CR240LA - In complience with NIST Database

# Zn1_Peak_Min = 307.54,  #307.54 For CR1000Y - Not in NIST Database , but shown in Sophie (LIBS Software)
# Zn1_Peak_Max =  307.64, #307.64 For CR1000Y - Not in NIST Database , but shown in Sophie (LIBS Software) 

# # Zn2_Peak_Min =  319.68, #For CR1000Y - Not in NIST Database , but shown in Sophie (LIBS Software)
# # Zn2_Peak_Max =  319.72, #For CR1000Y - Not in NIST Database , but shown in Sophie (LIBS Software)

Zn1_df = df_normalized[(df_normalized['wavelength'] >= Zn1_Peak_Min) & (df_normalized['wavelength'] <= Zn1_Peak_Max)]
Zn2_df = df_normalized[(df_normalized['wavelength'] >= Zn2_Peak_Min) & (df_normalized['wavelength'] <= Zn2_Peak_Max)]

Zn2_df

,wavelength,CR300LA_1,CR300LA_2,CR300LA_3,CR300LA_4,CR300LA_5,CR300LA_6,CR300LA_7,CR300LA_8,CR300LA_9,...,CR300LA_11,CR300LA_12,CR300LA_13,CR300LA_14,CR300LA_15,CR300LA_16,CR300LA_17,CR300LA_18,CR300LA_19,CR300LA_20
49244,467.807222,0.010607,0.002700,0.013576,0.017072,0.001822,0.008084,0.010272,0.031146,0.027795,...,0.004472,0.011203,0.007159,0.007402,0.031415,0.009680,0.018655,0.033116,-0.001290,-0.001697
49245,467.815507,-0.060736,0.004325,0.005452,0.023451,0.026097,0.030203,-0.033790,0.034099,0.033107,...,0.006813,0.026876,-0.006265,0.018904,0.027000,0.006074,-0.011839,0.021132,0.040950,0.007115
49246,467.823793,0.191123,0.001553,0.010992,0.020383,0.009052,0.003484,0.010336,0.028269,0.031362,...,0.000745,0.028070,0.006386,0.018744,0.018961,0.012856,0.010992,0.038478,0.005872,0.009936
49247,467.832078,0.078942,0.019169,0.025890,0.014002,0.015590,0.030900,-0.003399,-0.011838,-0.002952,...,0.021416,0.013516,-0.001326,0.006103,0.020121,0.009372,0.017708,0.036256,0.049147,0.013043
49248,467.840363,0.150758,0.007874,0.020998,-0.000900,0.016282,0.026003,0.044182,0.009344,0.089477,...,-0.003711,-0.009223,0.000056,-0.004082,0.017677,0.009852,0.028054,0.015346,0.028686,0.005593
49249,467.848648,0.079664,0.019351,0.013333,0.007228,0.010554,0.026154,0.002933,0.023489,0.013838,...,0.015994,0.033568,0.025302,0.003867,0.027559,0.019307,0.017854,0.028796,0.036641,0.015572
49250,467.856932,0.024930,0.009155,0.009179,0.017769,0.018739,0.000282,0.035706,-0.011249,0.025170,...,0.003792,0.018131,-0.002895,0.012905,0.009505,0.004525,0.004452,-0.020261,0.031527,0.008913
49251,467.865216,0.047783,0.015190,0.009725,0.013867,0.033586,-0.010790,-0.012768,0.053706,-0.008090,...,0.015107,-0.008914,0.006378,0.035947,0.020290,0.002317,0.004617,0.009109,0.048367,0.000306
49252,467.873501,-0.011643,0.020447,0.033755,-0.004934,0.012697,0.031304,0.084622,0.013183,-0.009380,...,0.015788,0.002974,0.017554,0.021610,0.021758,0.009406,-0.000705,0.017804,0.027113,0.021499
49253,467.881784,0.115362,0.025212,0.023381,0.013192,0.009243,0.077258,0.063171,0.011415,0.038933,...,0.015538,0.020656,0.020109,0.020370,0.011395,-0.004859,0.029362,0.042054,0.019077,0.026060


In [16]:
# Selected_Zn2_Plot = figure(title='Selected Zn2 plot', x_axis_label='Wavelength', y_axis_label='Intensity', width=300, height=500)

# for col_num, col_name in enumerate(Zn2_df.columns[1:], start=1):
#     intensity_values = Zn2_df[col_name]
#     Selected_Zn2_Plot.line(Zn2_df['wavelength'], intensity_values, line_width=2, color="red")

# show(Selected_Zn2_Plot)

In [17]:

# Load the dataframes
df1 = Zn1_df
df2 = Zn2_df

# Initialize lists to store peak intensities for each dataframe
peak_intensities_df1 = []
peak_intensities_df2 = []

# Determine x-axis values
x_values = list(range(5, 105, 5))

# Find the peak intensity for each column in each dataframe
for col_num in range(1, 21):  # Columns O  is the wavelength , dont use it
    peak_intensity_df1 = df1.iloc[:, col_num].max()  # Get the maximum intensity value for the column
    peak_intensity_df2 = df2.iloc[:, col_num].max()  # Get the maximum intensity value for the column
    peak_intensities_df1.append(peak_intensity_df1)
    peak_intensities_df2.append(peak_intensity_df2)

# Plotting
plot = figure(title='CR300LA Unpolished - Normalized by Matrix Peak i.e Fe 438.35', x_axis_label='Pulses', y_axis_label='Normalized Peak Intensity')
plot.scatter(x_values, peak_intensities_df1, legend_label='Zn 328.24', marker='+', color='blue', size=4)
plot.scatter(x_values, peak_intensities_df2, legend_label='Zn 468.02', marker='+', color='red', size=4)
plot.legend.location = "top_right"
plot.title.align = 'center'



#Set minor ticks
# plot.xaxis.ticker = x_values
# plot.yaxis.ticker = y_values
# # plot.yaxis.ticker = [i * 0.1 for i in range(11)]  # Adjust the range and step size as needed

# Set y-axis range
plot.y_range = Range1d(start=0, end=7)
plot.x_range = Range1d(start=(-1) , end =105)

# Connect markers with lines
for i in range(min(len(x_values), len(peak_intensities_df1)) - 1):
    plot.line([x_values[i], x_values[i+1]], [peak_intensities_df1[i], peak_intensities_df1[i+1]], line_color='blue')
    plot.line([x_values[i], x_values[i+1]], [peak_intensities_df2[i], peak_intensities_df2[i+1]], line_color='red')

# Show plot
show(plot)
